In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv('data/eiti-summary-company-payments.csv')

In [3]:
df = df[df['country'] == "Myanmar"]

In [4]:
df.head()

,created,changed,country,iso3,year,start_date,end_date,company_name,gfs_code,gfs_description,name_of_revenue_stream,currency_code,currency_rate,value_reported,value_reported_as_USD,reporting_url
31969,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1112-E1,"Ordinary taxes on income, profits and capital ...",Revenue Stream: 1112E1 [Ordinary taxes on inco...,MMK,960.5121,4.017128e+10,41822777.0,https://eiti.org/api/v1.0/organisation/31689
31970,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E1,Royalties,Revenue Stream: 1415E1 [Royalties] - Royalties,MMK,960.5121,1.407288e+11,146514378.0,https://eiti.org/api/v1.0/organisation/31689
31971,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,6.139777e+11,639219143.0,https://eiti.org/api/v1.0/organisation/31689
31972,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,1.208082e+11,125774746.0,https://eiti.org/api/v1.0/organisation/31689
31973,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,4.923598e+09,5126014.0,https://eiti.org/api/v1.0/organisation/31689


## Clean data

In [5]:
df["company_name"].unique()

array(['Petronas Carigali Myanmar Inc', 'TOTAL',
       'Daewoo International Corporation', 'PTT Exploration & Production',
       'Goldpetrol Co Ltd', 'MPRL E&P Pte Ltd', 'Nippon Oil', 'UNOCAL',
       'Chinnery Assets Ltd', 'SNOG Pte Ltd', 'Eternal Mining Company Ltd',
       'Kayah State Mining Production Co. (KMPC)',
       'Myanmar Pongpipat Company Limited', 'Nobel Gold',
       'Tha Byu Mining Co., Ltd.', 'Win Myint Mo Industries Co.,Ltd.',
       'DELCO (Developers Enterepreneurs Liaison Construction Organizers Limited',
       'GPS Joint Venture Co., Ltd.', 'Ngwe Yi Pale Mining Co., Ltd',
       'Max Myanmar Manufacturing Co.,',
       'Triple A Cement International Co., Ltd',
       'Myanmar Wanbao Mining Copper', 'Myanmar Yang Tse Copper Ltd',
       'CNMC Nickel Co Ltd. (MCNICO) (Tagaung Taung Nickel project)',
       'Great Genesis Gems Co., Ltd.', 'MYANMAR IMPERIAL JADE',
       'Richest Gems Co., Ltd', 'LINN LETT WIN YADANAR GEMS',
       'EVER WINNER GEMS', 'AUNG HEIN M

In [6]:
companies_info_df = pd.read_csv('data/eiti_summary_companies_cleaned.csv')

In [7]:
companies_info_df.head()

,Legal name,Identification,Company_name_cl,Identification_cl,Commodities
0,Petronas Carigali Myanmar Inc,NC,PETRONAS CARIGALI MYANMAR INC.(BRANCH OFFICE),16FC/2011-2012,Oil & Gas
1,TOTAL,24FC of 1992-1993,TOTAL E & P MYANMAR (YANGON BRANCH),24FC/1992-1993,Oil & Gas
2,Daewoo International Corporation,1 FC / 2002-2003,Posco Daewoo Corporation (Myanmar E&P Office),1FC/2002-2003,Oil & Gas
3,PTT Exploration & Production,84 FC,PTTEP International Ltd. (Yangon Branch),84FC/1995-1996,Oil & Gas
4,Goldpetrol Co Ltd,173 FC,Goldpetrol Co Ltd,173 FC,Oil & Gas


In [8]:
df = pd.merge(df, companies_info_df, left_on='company_name', right_on='Legal name')

In [9]:
df[df['Commodities'] == 'Gems & Jade']['name_of_revenue_stream'].unique()

array(['Revenue Stream: 1415E1 [Royalties] - Royalties',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Sale Split',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Emporium Fees / Sale Fees',
       'Revenue Stream: 1422E [Administrative fees for government services] - Supervision Fees',
       'Revenue Stream: 1415E5 [Other rent payments] - Other significant payments (&gt; 50,000 USD)'], dtype=object)

In [10]:
df[df['Commodities'] == 'Gems & Jade']['gfs_description'].unique()

array(['Royalties', 'Delivered/paid to state-owned enterprise(s)',
       'Administrative fees for government services', 'Other rent payments'], dtype=object)

In [11]:
df['revenue_stream_short'] = df['name_of_revenue_stream'].str.split(' - ').str[1]


In [12]:
df_gemsjade = df[df['Commodities'] == 'Gems & Jade']
df_gemsjade['type'] = 'entity'
df_gemsjade['target_type'] = ''

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [13]:
df_gemsjade.head()

,created,changed,country,iso3,year,start_date,end_date,company_name,gfs_code,gfs_description,...,value_reported_as_USD,reporting_url,Legal name,Identification,Company_name_cl,Identification_cl,Commodities,revenue_stream_short,type,target_type
76,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014","Great Genesis Gems Co., Ltd.",1415-E1,Royalties,...,4419839.0,https://eiti.org/api/v1.0/organisation/31716,"Great Genesis Gems Co., Ltd.",978/2005-2006,GREAT GENESIS GEMS COMPANY LIMITED,978/2005-2006,Gems & Jade,Royalties,entity,
77,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014","Great Genesis Gems Co., Ltd.",1415-E32,Delivered/paid to state-owned enterprise(s),...,15621836.0,https://eiti.org/api/v1.0/organisation/31716,"Great Genesis Gems Co., Ltd.",978/2005-2006,GREAT GENESIS GEMS COMPANY LIMITED,978/2005-2006,Gems & Jade,Sale Split,entity,
78,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014","Great Genesis Gems Co., Ltd.",1415-E32,Delivered/paid to state-owned enterprise(s),...,400000.0,https://eiti.org/api/v1.0/organisation/31716,"Great Genesis Gems Co., Ltd.",978/2005-2006,GREAT GENESIS GEMS COMPANY LIMITED,978/2005-2006,Gems & Jade,Emporium Fees / Sale Fees,entity,
79,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014","Great Genesis Gems Co., Ltd.",1422-E,Administrative fees for government services,...,397785.0,https://eiti.org/api/v1.0/organisation/31716,"Great Genesis Gems Co., Ltd.",978/2005-2006,GREAT GENESIS GEMS COMPANY LIMITED,978/2005-2006,Gems & Jade,Supervision Fees,entity,
80,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",MYANMAR IMPERIAL JADE,1415-E1,Royalties,...,3988599.0,https://eiti.org/api/v1.0/organisation/31717,MYANMAR IMPERIAL JADE,919 / 1996-1997,MYANMAR IMPERIAL JADE (GEMS & JEWELLERY) COMPA...,919/1996-1997,Gems & Jade,Royalties,entity,


In [14]:
company_totals = df_gemsjade.pivot_table(index=['Company_name_cl'], aggfunc='sum')['value_reported']
company_totals = company_totals.to_frame()
company_totals.rename(columns={'value_reported': 'total_payments'}, inplace=True)
company_totals.reset_index(level=0, inplace=True)
company_totals

,Company_name_cl,total_payments
0,(1.1.1) GEMS & JEWELLERY COMPANY LIMITED,8.420018e+09
1,AUNG HEIN MIN GEMS COMPANY LIMITED,2.196766e+09
2,AYAR JADE COMPANY LIMITED,7.016874e+09
3,AYEYAR YADANAR GEMS & JEWELLERY COMPANY LIMITED,1.083479e+09
4,"Ever Winner Gems Co., Ltd.",4.459207e+09
5,GREAT GENESIS GEMS COMPANY LIMITED,2.001655e+10
6,JADE MOUNTAIN COMPANY LIMITED,8.592584e+09
7,KHIN ZAW AUNG & BROTHERS GEMS AND JEWELLERY CO...,4.186186e+09
8,KYAING INTERNATIONAL GEMS COMPANY LIMITED,5.351262e+09
9,KYAUK SEINN NAGAR (GEMS) COMPANY LIMITED,9.122210e+09


In [15]:
df_gemsjade = pd.merge(df_gemsjade, company_totals, on='Company_name_cl')
df_gemsjade = df_gemsjade.sort_values(by=['total_payments'], ascending=False)

In [16]:
## Page 73 of 2013-14 Report. Section 6.2 Government receipts from companies not selected in the reconciliation scope

append_dict_others = [{'Company_name_cl': 'Companies not in EITI Reconciliation', 'type': 'entity', 
                       'revenue_stream_short': 'Royalties', 'value_reported': 111092756522 },
                      
                     {'Company_name_cl': 'Companies not in EITI Reconciliation', 'type': 'entity', 
                       'revenue_stream_short': 'Sale Split', 'value_reported': 51742994736 },
                      
                     {'Company_name_cl': 'Companies not in EITI Reconciliation', 'type': 'entity', 
                       'revenue_stream_short': 'Emporium Fees / Sale Fees', 'value_reported': 11401644588 },
                      
                     {'Company_name_cl': 'Companies not in EITI Reconciliation', 'type': 'entity', 
                       'revenue_stream_short': 'Supervision Fees', 'value_reported': 1246890624 },
                      
                     {'Company_name_cl': 'Companies not in EITI Reconciliation', 'type': 'entity', 
                       'revenue_stream_short': 'Other significant payments', 'value_reported': 2186525820 }]

others_df = pd.DataFrame(append_dict_others)
others_df['total_payments'] = others_df['value_reported']
df_gemsjade = pd.concat([df_gemsjade, others_df])
df_gemsjade.tail(10)

,Commodities,Company_name_cl,Identification,Identification_cl,Legal name,changed,company_name,country,created,currency_code,...,name_of_revenue_stream,reporting_url,revenue_stream_short,start_date,target_type,total_payments,type,value_reported,value_reported_as_USD,year
86,Gems & Jade,WAI FAMILY GEMS COMPANY LIMITED,174 / 2000-2001,174/2000-2001,WAI FAMILY GEMS,2017-02-13T07:00:42+0000,WAI FAMILY GEMS,Myanmar,2017-02-13T07:00:42+0000,MMK,...,Revenue Stream: 1415E32 [Delivered/paid to sta...,https://eiti.org/api/v1.0/organisation/31739,Emporium Fees / Sale Fees,"Apr 1, 2013",,1.109455e+09,entity,9.701789e+07,101006.0,2014.0
88,Gems & Jade,AYEYAR YADANAR GEMS & JEWELLERY COMPANY LIMITED,1532 /1996-1997,1532/1996-1997,AYEYAR YANDANAR GEMS,2017-02-13T07:00:42+0000,AYEYAR YANDANAR GEMS,Myanmar,2017-02-13T07:00:42+0000,MMK,...,Revenue Stream: 1415E32 [Delivered/paid to sta...,https://eiti.org/api/v1.0/organisation/31740,Emporium Fees / Sale Fees,"Apr 1, 2013",,1.083479e+09,entity,8.946155e+07,93139.0,2014.0
87,Gems & Jade,AYEYAR YADANAR GEMS & JEWELLERY COMPANY LIMITED,1532 /1996-1997,1532/1996-1997,AYEYAR YANDANAR GEMS,2017-02-13T07:00:42+0000,AYEYAR YANDANAR GEMS,Myanmar,2017-02-13T07:00:42+0000,MMK,...,Revenue Stream: 1415E1 [Royalties] - Royalties,https://eiti.org/api/v1.0/organisation/31740,Royalties,"Apr 1, 2013",,1.083479e+09,entity,9.940174e+08,1034883.0,2014.0
105,Gems & Jade,TREASURE WHITE LOTUS GEMS COMPANY LIMITED,273/2003-2004,273/2003-2004,TREASURE WHITE LOTUS GEMS,2017-02-13T07:00:42+0000,TREASURE WHITE LOTUS GEMS,Myanmar,2017-02-13T07:00:42+0000,MMK,...,Revenue Stream: 1415E1 [Royalties] - Royalties,https://eiti.org/api/v1.0/organisation/31745,Royalties,"Apr 1, 2013",,1.007072e+09,entity,9.234732e+08,961438.0,2014.0
106,Gems & Jade,TREASURE WHITE LOTUS GEMS COMPANY LIMITED,273/2003-2004,273/2003-2004,TREASURE WHITE LOTUS GEMS,2017-02-13T07:00:42+0000,TREASURE WHITE LOTUS GEMS,Myanmar,2017-02-13T07:00:42+0000,MMK,...,Revenue Stream: 1415E32 [Delivered/paid to sta...,https://eiti.org/api/v1.0/organisation/31745,Emporium Fees / Sale Fees,"Apr 1, 2013",,1.007072e+09,entity,8.359920e+07,87036.0,2014.0
0,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Royalties,NaN,NaN,1.110928e+11,entity,1.110928e+11,NaN,NaN
1,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sale Split,NaN,NaN,5.174299e+10,entity,5.174299e+10,NaN,NaN
2,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Emporium Fees / Sale Fees,NaN,NaN,1.140164e+10,entity,1.140164e+10,NaN,NaN
3,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Supervision Fees,NaN,NaN,1.246891e+09,entity,1.246891e+09,NaN,NaN
4,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Other significant payments,NaN,NaN,2.186526e+09,entity,2.186526e+09,NaN,NaN


In [17]:
df_gemsjade['revenue_stream_short'] = df_gemsjade['revenue_stream_short'].replace({'Other significant payments (&gt; 50,000 USD)': 'Other significant payments'})

## Remove negative payments for Sankey

In [18]:
df_gemsjade = df_gemsjade[df_gemsjade["value_reported"] > 0]
df_gemsjade['paid_to'] = "Myanmar Gems Enterprise"

In [19]:
df_gemsjade_summary = df_gemsjade[df_gemsjade['Company_name_cl'] != 'Companies not in EITI Reconciliation']
df_gemsjade_summary = df_gemsjade_summary.groupby(['revenue_stream_short','paid_to','target_type','type']).sum().reset_index()
df_gemsjade_summary['Company_name_cl'] = 'Companies in EITI Reconciliation'
df_gemsjade_summary = df_gemsjade[df_gemsjade['Company_name_cl'] == 'Companies not in EITI Reconciliation'] \
                        .append(df_gemsjade_summary)
df_gemsjade_summary

,Commodities,Company_name_cl,Identification,Identification_cl,Legal name,changed,company_name,country,created,currency_code,...,paid_to,reporting_url,revenue_stream_short,start_date,target_type,total_payments,type,value_reported,value_reported_as_USD,year
0,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Royalties,NaN,NaN,1.110928e+11,entity,1.110928e+11,NaN,NaN
1,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Sale Split,NaN,NaN,5.174299e+10,entity,5.174299e+10,NaN,NaN
2,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Emporium Fees / Sale Fees,NaN,NaN,1.140164e+10,entity,1.140164e+10,NaN,NaN
3,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Supervision Fees,NaN,NaN,1.246891e+09,entity,1.246891e+09,NaN,NaN
4,NaN,Companies not in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Other significant payments,NaN,NaN,2.186526e+09,entity,2.186526e+09,NaN,NaN
0,NaN,Companies in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Emporium Fees / Sale Fees,NaN,,2.025259e+11,entity,6.127875e+09,6379800.0,60420.0
1,NaN,Companies in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Other significant payments,NaN,,1.893933e+09,entity,8.955549e+07,93237.0,2014.0
2,NaN,Companies in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Royalties,NaN,,2.025259e+11,entity,5.893137e+10,61354119.0,60420.0
3,NaN,Companies in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Sale Split,NaN,,1.864555e+11,entity,1.341873e+11,139703870.0,46322.0
4,NaN,Companies in EITI Reconciliation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myanmar Gems Enterprise,NaN,Supervision Fees,NaN,,1.864555e+11,entity,3.189849e+09,3320985.0,46322.0


## Prepare Source-Target-Value dataframe

In [20]:
links_companies = pd.DataFrame(columns=['source','target','value','type'])

In [21]:


to_append = df_gemsjade.groupby(['revenue_stream_short','paid_to'],as_index=False)['type','value_reported','total_payments'].sum()
to_append.rename(columns = {'revenue_stream_short':'source', 'value_reported' : 'value', 'paid_to': 'target'}, inplace = True)

to_append = to_append.sort_values(by=['value'], ascending = False)
to_append['target_type'] = 'entity'

links_companies = pd.concat([links_companies,to_append])

print(to_append['value'].sum())
links_companies

380196722113.0


,source,target,target_type,total_payments,type,value
3,Sale Split,Myanmar Gems Enterprise,entity,2.381985e+11,NaN,1.859303e+11
2,Royalties,Myanmar Gems Enterprise,entity,3.136187e+11,NaN,1.700241e+11
0,Emporium Fees / Sale Fees,Myanmar Gems Enterprise,entity,2.139276e+11,NaN,1.752952e+10
4,Supervision Fees,Myanmar Gems Enterprise,entity,1.877024e+11,NaN,4.436739e+09
1,Other significant payments,Myanmar Gems Enterprise,entity,4.080459e+09,NaN,2.276081e+09


In [22]:
## Page 131 of 2013-14 Report. Appendix 8: SOEs reconciliation sheets

append_dict_transfers = [{'source': 'Myanmar Gems Enterprise', 'type': 'entity',
                       'target': 'State Contribution', 'value': 48367022000 },
                         
                         {'source': 'Myanmar Gems Enterprise', 'type': 'entity', 
                       'target': 'Transfers to Other Accounts', 'value': 195516457782 },
                         
                         {'source': 'Myanmar Gems Enterprise', 'type': 'entity', 
                       'target': 'Commercial Tax', 'value': 21499000 },
                         
                         {'source': 'Myanmar Gems Enterprise', 'type': 'entity', 
                       'target': 'Corporate Income Tax', 'value': 60458777000 },
                         
                         {'source': 'Myanmar Gems Enterprise', 'type': 'entity', 
                       'target': 'Other material transfers', 'value': 119588538585 },
                         
                         {'source': 'State Contribution', 'target_type': 'entity',
                       'target': 'Ministry of Finance', 'value': 48367022000 },
                         
                         {'source': 'Corporate Income Tax', 
                       'target': 'Internal Revenue Department', 'value': 60458777000 },
                         
                         {'source': 'Commercial Tax', 
                       'target': 'Internal Revenue Department', 'value': 21499000 },
                        
                         {'source': 'Internal Revenue Department', 'type': 'entity','target_type': 'entity',
                       'target': 'Ministry of Finance', 'value': 60480276000 }]

#append_dict_transfers_df = pd.DataFrame(append_dict_transfers)
#links = pd.concat([links, append_dict_transfers_df])
#links

append_dict_transfers_df = pd.DataFrame(append_dict_transfers)

links_summary = pd.concat([links_companies, append_dict_transfers_df])
links_govt = append_dict_transfers_df

In [23]:
'''
to_append = df_gemsjade.groupby(['revenue_stream_short','Company_name_cl','type'],as_index=False)['value_reported','value_reported_as_USD','total_payments'].sum()
to_append.rename(columns = {'Company_name_cl':'source','revenue_stream_short':'target', 'value_reported' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['type','total_payments','value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
#links
to_append
'''

to_append = df_gemsjade.groupby(['revenue_stream_short','Company_name_cl','type'],as_index=False) \
    ['value_reported','total_payments'] \
    .agg({'value_reported':sum,'total_payments':'first'})
to_append.rename(columns = {'Company_name_cl':'source','revenue_stream_short':'target', 'value_reported' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['total_payments'], ascending = False)
links_companies = pd.concat([links_companies,to_append])

print(to_append['value'].sum())
#links
to_append

380196722113.0


,target,source,type,value,total_payments
37,Royalties,Companies not in EITI Reconciliation,entity,1.110928e+11,1.110928e+11
67,Sale Split,Companies not in EITI Reconciliation,entity,5.174299e+10,5.174299e+10
39,Royalties,GREAT GENESIS GEMS COMPANY LIMITED,entity,4.245309e+09,2.001655e+10
6,Emporium Fees / Sale Fees,GREAT GENESIS GEMS COMPANY LIMITED,entity,3.842051e+08,2.001655e+10
92,Supervision Fees,GREAT GENESIS GEMS COMPANY LIMITED,entity,3.820778e+08,2.001655e+10
68,Sale Split,GREAT GENESIS GEMS COMPANY LIMITED,entity,1.500496e+10,2.001655e+10
24,Emporium Fees / Sale Fees,WAI AUNG GABAR GEMS COMPANY LIMITED,entity,4.170156e+08,1.807804e+10
108,Supervision Fees,WAI AUNG GABAR GEMS COMPANY LIMITED,entity,3.212483e+08,1.807804e+10
57,Royalties,WAI AUNG GABAR GEMS COMPANY LIMITED,entity,3.942465e+09,1.807804e+10
84,Sale Split,WAI AUNG GABAR GEMS COMPANY LIMITED,entity,1.339731e+10,1.807804e+10


In [24]:
to_append = df_gemsjade_summary.groupby(['revenue_stream_short','Company_name_cl','type'],as_index=False) \
    ['value_reported','total_payments'] \
    .agg({'value_reported':sum,'total_payments':'first'})
to_append.rename(columns = {'Company_name_cl':'source','revenue_stream_short':'target', 'value_reported' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['total_payments'], ascending = False)
links_summary = pd.concat([links_summary,to_append])
links_summary

,source,target,target_type,total_payments,type,value
3,Sale Split,Myanmar Gems Enterprise,entity,2.381985e+11,NaN,1.859303e+11
2,Royalties,Myanmar Gems Enterprise,entity,3.136187e+11,NaN,1.700241e+11
0,Emporium Fees / Sale Fees,Myanmar Gems Enterprise,entity,2.139276e+11,NaN,1.752952e+10
4,Supervision Fees,Myanmar Gems Enterprise,entity,1.877024e+11,NaN,4.436739e+09
1,Other significant payments,Myanmar Gems Enterprise,entity,4.080459e+09,NaN,2.276081e+09
0,Myanmar Gems Enterprise,State Contribution,NaN,NaN,entity,4.836702e+10
1,Myanmar Gems Enterprise,Transfers to Other Accounts,NaN,NaN,entity,1.955165e+11
2,Myanmar Gems Enterprise,Commercial Tax,NaN,NaN,entity,2.149900e+07
3,Myanmar Gems Enterprise,Corporate Income Tax,NaN,NaN,entity,6.045878e+10
4,Myanmar Gems Enterprise,Other material transfers,NaN,NaN,entity,1.195885e+11


In [25]:
#to_append = df.groupby(['Payment Type','Reporting Company'],as_index=False)['Value (USD)'].sum()
#to_append.rename(columns = {'Payment Type':'source','Reporting Company':'target', 'Value (USD)' : 'value'}, inplace = True)
#to_append = to_append.sort_values(by=['value'], ascending = False)
#links = pd.concat([links,to_append])

#print(to_append['value'].sum())
#links

In [26]:
'''
unique_source = links['source'].unique()
unique_targets = links['target'].unique()

unique_source = pd.merge(pd.DataFrame(unique_source), links, left_on=0, right_on='source', how='left')
unique_source = unique_source.filter([0,'type'])
unique_targets = pd.merge(pd.DataFrame(unique_targets), links, left_on=0, right_on='target', how='left')
unique_targets = unique_targets.filter([0,'target_type'])
unique_targets.rename(columns = {'target_type':'type'}, inplace = True)

unique_list = pd.concat([unique_source[0], unique_targets[0]]).unique()

unique_list = pd.merge(pd.DataFrame(unique_list), \
                       pd.concat([unique_source, unique_targets]), left_on=0, right_on=0, how='left')

unique_list.drop_duplicates(subset=0, keep='first', inplace=True)

replace_dict = {k: v for v, k in enumerate(unique_list[0])}
unique_list
#replace_dict
'''

def prep_nodes_links(links):
    unique_source = links['source'].unique()
    unique_targets = links['target'].unique()

    unique_source = pd.merge(pd.DataFrame(unique_source), links, left_on=0, right_on='source', how='left')
    unique_source = unique_source.filter([0,'type'])
    unique_targets = pd.merge(pd.DataFrame(unique_targets), links, left_on=0, right_on='target', how='left')
    unique_targets = unique_targets.filter([0,'target_type'])
    unique_targets.rename(columns = {'target_type':'type'}, inplace = True)

    unique_list = pd.concat([unique_source[0], unique_targets[0]]).unique()

    unique_list = pd.merge(pd.DataFrame(unique_list), \
                           pd.concat([unique_source, unique_targets]), left_on=0, right_on=0, how='left')

    unique_list.drop_duplicates(subset=0, keep='first', inplace=True)

    replace_dict = {k: v for v, k in enumerate(unique_list[0])}
    unique_list
    return [unique_list,replace_dict]

In [27]:
###links_replaced = links.replace({"source": replace_dict,"target": replace_dict})

[unique_list_summary,replace_dict_summary] = prep_nodes_links(links_summary)
[unique_list_companies,replace_dict_companies] = prep_nodes_links(links_companies)
[unique_list_govt,replace_dict_govt] = prep_nodes_links(links_govt)

In [28]:
def write_nodes_links(filename,unique_list,replace_dict,links):
    links_replaced = links.replace({"source": replace_dict,"target": replace_dict})
    nodes = pd.DataFrame(unique_list)
    nodes.rename(columns = {0:'name'}, inplace = True)
    nodes_json= pd.DataFrame(nodes).to_json(orient='records')
    links_json= pd.DataFrame(links_replaced).to_json(orient='records')
    
    data = { 'links' : json.loads(links_json), 'nodes' : json.loads(nodes_json) }
    data_json = json.dumps(data)
    data_json = data_json.replace("\\","")
    #print(data_json)
    #with open('sankey_data.json', 'w') as outfile:
    #    json.dump(data_json, outfile)

    text_file = open(filename + ".json", "w")
    text_file.write(data_json)
    text_file.close()

In [29]:
write_nodes_links("sankey_data_2013-14_summary",unique_list_summary,replace_dict_summary,links_summary)
write_nodes_links("sankey_data_2013-14_companies",unique_list_companies,replace_dict_companies,links_companies)
write_nodes_links("sankey_data_2013-14_govt",unique_list_govt,replace_dict_govt,links_govt)

In [30]:
#links_replaced

In [31]:
#nodes = pd.DataFrame(unique_list)
#nodes.rename(columns = {0:'name'}, inplace = True)

In [32]:
#nodes_json= pd.DataFrame(nodes).to_json(orient='records')
#nodes_json 

In [33]:
#links_json= pd.DataFrame(links_replaced).to_json(orient='records')
#links_json 

In [34]:
#data = { 'links' : json.loads(links_json), 'nodes' : json.loads(nodes_json) }
#data_json = json.dumps(data)
#data_json = data_json.replace("\\","")
#print(data_json)
#with open('sankey_data.json', 'w') as outfile:
#    json.dump(data_json, outfile)

#text_file = open("sankey_data_2013-14.json", "w")
#text_file.write(data_json)
#text_file.close()